In [78]:
import numpy as np
import pandas as pd
from numpy import random
from numpy.random import randn
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
import kf_book.book_plots as bp

df = pd.read_csv('kf3105.csv')
uwb_x = df.uwb_x

In [31]:
len(uwb_x)

472

In [8]:
def predict(pos, movement):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior

In [77]:
import kf_book.kf_internal as kf_internal
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '(={:.3f}, 2={:.3f})'.format(s[0], s[1])
# note: A large variance implies that confidence is very low, 
# so the filter estimates the position to be very close to the measurement
# specific to the robot
process_var = 0.028 ** 2
sensor_var = 0.077 ** 2
# initial uwb_x: 2
x = gaussian(uwb_x[0], sensor_var) # robot's position(x-coordinate)
velocity = 0.5
# if changed to 0.1 the kf precision will be lower
dt = 0.01
process_model = gaussian(velocity*dt, process_var)
# measurements are uwb_x
# Kalman filter
print('    PREDICT\t\tMEASURE    UPDATE')
print('  x\t   var\t\tz\t  x\t  var')
# perform kf on measurement
for z in uwb_x:
    prior = predict(x, process_model)
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood)
    
    kf_internal.print_gh(prior, x, z)
print('final estimate X-coordinate:\t{:10.3f} m'.format(x.mean))
print('tag final position:\t\t{:10.3f} m'.format(uwb_x.iloc[-1]))

    PREDICT		MEASURE    UPDATE
  x	   var		z	  x	  var
  2.005    0.007	2.000	  2.002   0.003
  2.007    0.004	2.000	  2.004   0.002
  2.009    0.003	2.000	  2.006   0.002
  2.011    0.003	2.000	  2.008   0.002
  2.013    0.003	2.000	  2.009   0.002
  2.014    0.003	2.000	  2.009   0.002
  2.014    0.003	2.000	  2.010   0.002
  2.015    0.003	2.000	  2.010   0.002
  2.015    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	2.000	  2.011   0.002
  2.016    0.003	1.990	  2.008   0.002
  2.013    0.003	1.990	  2.006   0.002
  2.011    0.003	1.990	  2.005   0.002
  2.010  

  3.302    0.003	3.260	  3.290   0.002
  3.295    0.003	3.260	  3.284   0.002
  3.289    0.003	3.260	  3.280   0.002
  3.285    0.003	3.260	  3.278   0.002
  3.283    0.003	3.260	  3.276   0.002
  3.281    0.003	3.260	  3.274   0.002
  3.279    0.003	3.270	  3.277   0.002
  3.282    0.003	3.270	  3.278   0.002
  3.283    0.003	3.270	  3.279   0.002
  3.284    0.003	3.270	  3.280   0.002
  3.285    0.003	3.270	  3.280   0.002
  3.285    0.003	3.270	  3.281   0.002
  3.286    0.003	3.270	  3.281   0.002
  3.286    0.003	3.270	  3.281   0.002
  3.286    0.003	3.270	  3.281   0.002
  3.286    0.003	3.270	  3.281   0.002
  3.286    0.003	3.290	  3.287   0.002
  3.292    0.003	3.290	  3.292   0.002
  3.297    0.003	3.290	  3.295   0.002
  3.300    0.003	3.290	  3.297   0.002
  3.302    0.003	3.290	  3.298   0.002
  3.303    0.003	3.290	  3.299   0.002
  3.304    0.003	3.290	  3.300   0.002
  3.305    0.003	3.290	  3.300   0.002
  3.305    0.003	3.290	  3.301   0.002
  3.306    0.003	3.290	  

In [69]:
from kf_book import book_plots as book_plots
from ipywidgets import interact
from ipywidgets.widgets import IntSlider

# save output in these lists for plotting
xs, predictions = [], []

# process_model = gaussian(velocity, process_var) 

# perform Kalman filter
# x = gaussian(0., 20.**2)
for z in uwb_x:    
    prior = predict(x, process_model)
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood)

    # save results
    predictions.append(prior.mean)
    xs.append(x.mean)

def plot_filter(step):
    plt.cla()
    step -= 1
    i = step // 3 + 1
 
    book_plots.plot_predictions(predictions[:i])    
    zs = uwb_x
    if step % 3 == 0:
        book_plots.plot_measurements(zs[:i-1])
        book_plots.plot_filter(xs[:i-1])
    elif step % 3 == 1:
        book_plots.plot_measurements(zs[:i])
        book_plots.plot_filter(xs[:i-1])
    else:
        book_plots.plot_measurements(zs[:i])
        book_plots.plot_filter(xs[:i])

    plt.xlim(-5, 475)
    plt.ylim(1.5, 4.5)
    plt.xlabel("Number of measurements")
    plt.ylabel("X-coordinate (m)")
#     loc=2
    plt.legend(loc=2);
#     plt.savefig('kalman.pdf', dpi=300)
interact(plot_filter, step=IntSlider(value=1, min=1, max=len(predictions)*3));

interactive(children=(IntSlider(value=1, description='step', max=1416, min=1), Output()), _dom_classes=('widge…